## Deliverable 2. Create a Customer Travel Destinations Map.

In [86]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
import sys
sys.path.append('C:/Users/Bryan/DataClass/World_Weather_Analysis')
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [87]:
#Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jamestown,US,42.0970,-79.2353,85.01,48,1,3.00,clear sky
1,1,Georgetown,MY,5.4112,100.3354,78.89,93,40,3.44,scattered clouds
2,2,Hamilton,US,39.1834,-84.5333,87.15,65,14,1.01,few clouds
3,3,Huarmey,PE,-10.0681,-78.1522,63.30,83,16,10.92,few clouds
4,4,Tiksi,RU,71.6872,128.8694,44.78,75,99,4.97,overcast clouds


In [88]:
#Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 90


In [89]:
#Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jamestown,US,42.0970,-79.2353,85.01,48,1,3.00,clear sky
2,2,Hamilton,US,39.1834,-84.5333,87.15,65,14,1.01,few clouds
19,19,Hithadhoo,MV,-0.6000,73.0833,81.16,72,97,17.58,overcast clouds
21,21,Lufilufi,WS,-13.8745,-171.5986,87.58,77,35,24.79,scattered clouds
31,31,Vaini,TO,-21.2000,-175.2000,80.76,94,40,10.36,scattered clouds
34,34,Raipur,IN,21.2333,81.6333,81.68,75,100,2.95,overcast clouds
37,37,Kapaa,US,22.0752,-159.3190,83.19,70,5,4.00,clear sky
47,47,Ixtapa,MX,20.7000,-105.2000,85.23,66,86,1.99,overcast clouds
48,48,Albany,US,42.6001,-73.9662,81.57,56,4,2.46,clear sky
54,54,Xuanhua,CN,40.6103,115.0447,80.46,69,81,4.79,broken clouds


In [90]:
#Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                132
City                   132
Country                132
Lat                    132
Lng                    132
Max Temp               132
Humidity               132
Cloudiness             132
Wind Speed             132
Current Description    132
dtype: int64

In [91]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Jamestown,US,85.01,clear sky,42.0970,-79.2353,
2,Hamilton,US,87.15,few clouds,39.1834,-84.5333,
19,Hithadhoo,MV,81.16,overcast clouds,-0.6000,73.0833,
21,Lufilufi,WS,87.58,scattered clouds,-13.8745,-171.5986,
31,Vaini,TO,80.76,scattered clouds,-21.2000,-175.2000,
34,Raipur,IN,81.68,overcast clouds,21.2333,81.6333,
37,Kapaa,US,83.19,clear sky,22.0752,-159.3190,
47,Ixtapa,MX,85.23,overcast clouds,20.7000,-105.2000,
48,Albany,US,81.57,clear sky,42.6001,-73.9662,
54,Xuanhua,CN,80.46,broken clouds,40.6103,115.0447,


In [92]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    "location": "0, 0"
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [93]:
#Turn empty string cells into Null
clean_hotel_df = hotel_df
clean_hotel_df["Hotel Name"].replace("", np.nan, inplace=True)
# Drop the rows where there is no Hotel Name.
clean_hotel_df.dropna(axis = 0, how = 'any', thresh=None, subset=None,inplace=True)
#check to see if rows deleted
clean_hotel_df.count()

City                   127
Country                127
Max Temp               127
Current Description    127
Lat                    127
Lng                    127
Hotel Name             127
dtype: int64

In [94]:
# Create the output file (CSV).
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
#Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [95]:
#Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

#Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

#Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [96]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))